In [14]:
import os
import numpy as np
import face_recognition as fr
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import joblib

In [15]:
# Defina o caminho para a pasta contendo as imagens
image_folder_train = r'D:\FIAP\2024\1_Semestre\AI ENGENEERING, COGNITIVE AND SEMANTIC COMPUTATION & IOT\CP\KenAI\Reconhecimento_pessoa\images\Larissa'
image_folder_test = r'D:\FIAP\2024\1_Semestre\AI ENGENEERING, COGNITIVE AND SEMANTIC COMPUTATION & IOT\CP\KenAI\Reconhecimento_pessoa\images\todos'

# Obtenha uma lista de todos os arquivos na pasta
image_files_train = [f for f in os.listdir(image_folder_train) if os.path.isfile(os.path.join(image_folder_train, f))]
image_files_test = [f for f in os.listdir(image_folder_test) if os.path.isfile(os.path.join(image_folder_test, f))]


In [16]:
# Função para carregar e preprocessar as imagens usando face encodings
def load_and_preprocess_images(image_folder, image_files):
    encodings = []
    labels = []
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        image = fr.load_image_file(image_path)  # Carrega a imagem
        face_locations = fr.face_locations(image)  # Encontra as localizações dos rostos
        if face_locations:
            face_encoding = fr.face_encodings(image, known_face_locations=face_locations)[0]
            encodings.append(face_encoding)
            labels.append(image_file)  # Assume que o nome do arquivo é o label
    encodings = np.array(encodings)
    return encodings, labels


In [17]:
x_train, labels_train = load_and_preprocess_images(image_folder_train, image_files_train)
x_test, labels_test = load_and_preprocess_images(image_folder_test, image_files_test)


KeyboardInterrupt: 

In [ ]:
# Encode the labels to numerical values
label_encoder = LabelEncoder()
label_encoder.fit(labels_train)  # Ajusta o LabelEncoder apenas com os rótulos de treinamento

# Filtra os dados de teste para incluir apenas rótulos presentes nos dados de treinamento
filtered_indices = [i for i, label in enumerate(labels_test) if label in label_encoder.classes_]
x_test_filtered = x_test[filtered_indices]
labels_test_filtered = [labels_test[i] for i in filtered_indices]


ValueError: y contains previously unseen labels: 'mulher1.jpg'

In [ ]:
y_test = label_encoder.transform(labels_test_filtered)

# Treinar o modelo RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, label_encoder.transform(labels_train))  # Transforma rótulos de treinamento para valores numéricos


In [ ]:
# Fazer previsões
y_pred = model.predict(x_test_filtered)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


In [ ]:
# Função para plotar a importância das features
from matplotlib import pyplot as plt

def plot_feature_importance(model):
    plt.figure(figsize=(10, 6))
    feature_importance = model.feature_importances_
    indices = np.argsort(feature_importance)[::-1]
    plt.bar(range(len(feature_importance)), feature_importance[indices])
    plt.xlabel('Feature')
    plt.ylabel('Importance')
    plt.title('Feature Importance')
    plt.show()

plot_feature_importance(model)

In [ ]:
model_path = r'D:\FIAP\2024\1_Semestre\AI ENGENEERING, COGNITIVE AND SEMANTIC COMPUTATION & IOT\CP\KenAI\Reconhecimento_pessoa\random_forest_model.pkl'
joblib.dump(model, model_path)